# Redis
---
- [Installation](#Installation)
- [Quick Started](#Getting-Started)
- [Data Types](#Data-Types)
 - [Strings](#Strings)
 - [Lists](#Lists)
 - [Sets](#Sets)
 - [Hashes](#Hashes)
 - [Sorted sets](#Sorted-sets)


# Installation
## [redis-py](https://github.com/andymccurdy/redis-py)

In [1]:
!pip install redis

In [ ]:
#使得Jupyter對獨占一行的所有變量或者語句都自動顯示
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Getting Started

In [1]:
import redis
r = redis.StrictRedis(host='localhost', port=6379, db=0)

In [25]:
r.set('foo', 'bar')
r.exists('bar')
r.get('foo')
r.delete('foo')

True

False

b'bar'

1

In [32]:
#Redis expires: keys with limited time to live
import time
r.set('exTest' , 'one second',ex=1)
r.get('exTest')
time.sleep(1)
r.get('exTest') #None
#same
r.set('key','value')
r.expire('key',1)
r.ttl('key') # check the remaining time to live for the key.
time.sleep(1)
r.get('key') #None

True

b'one second'

True

True

1

# Data Types
topics:https://redis.io/topics/data-types
---
## Strings
- binary safe(any kind of data)
- max 512 Megabytes

Commands:
- GET,SET
- INCR, INCRBY, DECR, DECRBY
- APPEND, STRLEN
- MGET, MSET

In [21]:
r.set('mykey', 100)
r.get('mykey')
#SET will replace any existing value already stored into the key
r.set('mykey', 111)
r.get('mykey')
# fail if the key already exists
r.set('mykey', 123, nx=True)
print(r.set('mykey', 123, nx=True)) #fail return None (nil)
r.get('mykey')
# it only succeed if the key already exists
r.set('mykey', 123, xx=True)
r.get('mykey')

True

b'100'

True

b'111'

None


b'111'

True

b'123'

In [39]:
#
r.get('mykey')
r.incr('mykey')
r.get('mykey')
#
r.incrby('mykey', 10)
r.get('mykey')
#
r.set('key', 'value')
try:
    r.incr('key')
except Exception as e:
    e

b'190'

191

b'191'

201

b'201'

True

redis.exceptions.ResponseError('value is not an integer or out of range')

In [45]:
#append
r.set('key', 'Hello')
r.append('key', ' World!')
r.get('key')
#strlen
r.strlen('key')

True

12

b'Hello World!'

12

In [48]:
r.mset({'key':'value' , 'key2':'value2'})
r.mget(['key', 'key2'])
#
r.mget(r.keys('*'))

True

[b'value', b'value2']

[b'value', b'201', b'value2']

In [85]:
r.getset('key','value_bar')

b'value'

In [86]:
r.get('key')

b'value_bar'

## Lists
- Lists of string, sorted by insertion order.
- Redis lists are implemented via Linked Lists.
- The max length of a list is $2^{32} - 1$ elements (4294967295, more than 4 billion of elements per list).

---
Commands:
- LPUSH, RPUSH
- LPOP, RPOP
- LRANGE
- LINDEX
- LREM
- LSET
- LINSERT
- RPOPLPUSH


## Sets
---

---
Commands:
- SADD, SREM
- SMSMBERS
- SISMEMBER
- SDIFF, SINTER, SUNION
- SPOP
- SRANDOMEMBER

In [74]:
r.sadd('tag_set','java')
r.sadd('tag_set', 'javascript', 'java', 'mysql',['java'])
r.smembers('tag_set')

1

3

{b"['java']", b'java', b'javascript', b'mysql'}

In [75]:
r.sismember('tag_set', 'redis')
r.sismember('tag_set', 'java')

False

True

In [76]:
r.delete('tag_set')

1

In [77]:
r.sadd('setA', 1,2,3,4,5)
r.sadd('setB', 4,5,6,7,8,9)

5

6

In [83]:
#differetial
r.sdiff('setA', 'setB')
r.sdiff('setB', 'setA')
#intersection
r.sinter('setA','setB')
#union and store
r.sunionstore('setC','setA','setB')
r.smembers('setC')

{b'1', b'2', b'3'}

{b'6', b'7', b'8', b'9'}

{b'4', b'5'}

9

{b'1', b'2', b'3', b'4', b'5', b'6', b'7', b'8', b'9'}

In [84]:
# random members
r.srandmember('setC')
r.srandmember('setC', number=2)
# pop 
r.spop('setC')
r.smembers('setC')

b'7'

[b'5', b'8']

b'8'

{b'1', b'2', b'3', b'4', b'5', b'6', b'7', b'9'}

## Hashes
Redis Hashes are maps between string fields and string values.  
Commands:  
- HGET, HSET
- HGETALL
- HMSET, HMGET
- HEXISTS

In [52]:
r.hset('name','key','value')
r.hset('name','key2','value2')
r.hget('name', 'key')
r.hgetall('name')

0

1

b'value'

{b'key': b'value', b'key2': b'value2'}

In [59]:
r.hmset('name',{'key':'value','key2':'value2', 'key3':'valuse3'})
r.hmget('name',['key','key3'])

True

[b'value', b'valuse3']

In [62]:
r.hexists('name','key4')

False

In [64]:
r.hkeys('name')
r.hvals('name')

[b'key', b'key2', b'key3']

[b'value', b'value2', b'valuse3']

## Sorted sets
-  similar to a mix between a Set and a Hash 
-  every element in a sorted set is associated with a floating point value, called the score
---
Commands:
- ZADD
- ZRANGE
- ZRANGEBYSCORE
- ZINCRBY
- ZCARD, ZCOUNT
- ZREM, ZREMRANGEBYSCORE
- ZRANK, ZREVRANK

In [131]:
r.zadd('zset', 87, 'Tom', 67, 'Peter', David=100)

2

In [127]:
r.zadd('zset', 88, 'Tom')

0

In [128]:
r.zrange('zset', 0 , -1)
r.zrange('zset', 0 , -1, desc=True)
r.zrange('zset', 0 , -1, withscores=True)

[b'Peter', b'Tom', b'David']

[b'David', b'Tom', b'Peter']

[(b'Peter', 67.0), (b'Tom', 88.0), (b'David', 100.0)]

In [129]:
r.zrangebyscore('zset' , 80, 100)
r.zrangebyscore('zset', 80, '(100' )
r.zrangebyscore('zset', 80, '+inf' , withscores=True)
r.zrangebyscore('zset', '-inf', '+inf')
r.zrangebyscore('zset', '-inf', '+inf', start=1, num=3)

[b'Tom', b'David']

[b'Tom']

[(b'Tom', 88.0), (b'David', 100.0)]

[b'Peter', b'Tom', b'David']

[b'Tom', b'David']

In [130]:
#zrem
r.zrem('zset', 'Peter')
r.zrange('zset', 0 , -1)
#zremrangebyscore
r.zremrangebyscore('zset', min=80, max=90)
r.zrange('zset', 0 , -1)

1

[b'Tom', b'David']

1

[b'David']

In [132]:
r.zadd('zset', 88, 'Tom2', 65, 'Peter2', David2=99)

3

In [133]:
r.zrank('zset', 'Tom2')

3